In [1]:
from mxnet import nd, gluon, autograd
import matplotlib.pyplot as plt

In [2]:
def transform(x, y):
    return x.astype('float32')/255, y.astype('float32')

In [3]:
mnistTrain = gluon.data.vision.FashionMNIST(root='/workdir/mnist/', train=True, transform=transform)
mnistTest = gluon.data.vision.FashionMNIST(root='/workdir/mnist/', train=False, transform=transform)

In [4]:
batchSize = 256
trainData = gluon.data.DataLoader(mnistTrain, batchSize, shuffle=True)
testData = gluon.data.DataLoader(mnistTest, batchSize, shuffle=False)

In [5]:
net = gluon.nn.Sequential()

In [6]:
net.add(gluon.nn.Flatten())
net.add(gluon.nn.Dense(10))

with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(10))

In [7]:
net.initialize()

In [8]:
dense = net[0]
dense.params

flatten0_ (

)

In [9]:
dense = net[1]
dense.params

dense0_ (
  Parameter dense0_weight (shape=(10, 0), dtype=<class 'numpy.float32'>)
  Parameter dense0_bias (shape=(10,), dtype=<class 'numpy.float32'>)
)

In [10]:
net.__len__()

2

In [11]:
net.collect_params()

sequential0_ (
  Parameter dense0_weight (shape=(10, 0), dtype=<class 'numpy.float32'>)
  Parameter dense0_bias (shape=(10,), dtype=<class 'numpy.float32'>)
)

In [12]:
softmaxCrossEntropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [13]:
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.1})

In [14]:
epochs = 5

In [15]:
def accuracy(yhat, y):
    return nd.mean(yhat.argmax(axis=1)==y).asscalar()

In [16]:
def evaluateAcc(dataIterator, net):
    acc = 0.
    for x, y in dataIterator:
        yhat = net(x)
        acc += accuracy(yhat, y)
    return acc / len(dataIterator)

In [17]:
for epoch in range(epochs):
    trainLoss = 0.
    trainAcc = 0.
    for x, y in trainData:
        with autograd.record():
            yhat = net(x)
            loss = softmaxCrossEntropy(yhat, y)
        loss.backward()
        trainer.step(batchSize)

        trainLoss += nd.mean(loss).asscalar()
        trainAcc += accuracy(yhat, y)
    testAcc = evaluateAcc(testData, net)
    print('Epoch %d. Loss: %f, Train acc: %f, Test acc %f' % (
        epoch, trainLoss/len(trainData), trainAcc/len(trainData), testAcc))

Epoch 0. Loss: 0.790322, Train acc: 0.744021, Test acc 0.798242
Epoch 1. Loss: 0.575843, Train acc: 0.809824, Test acc 0.822949
Epoch 2. Loss: 0.531281, Train acc: 0.822762, Test acc 0.827832
Epoch 3. Loss: 0.506007, Train acc: 0.830724, Test acc 0.827930
Epoch 4. Loss: 0.490443, Train acc: 0.834164, Test acc 0.838770
